In [1]:
!pip install -U odp-sdk --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azure-cli-core 2.68.0 requires microsoft-security-utilities-secret-masker~=1.0.0b2, which is not installed.
azure-cli-core 2.68.0 requires argcomplete~=3.5.2, but you have argcomplete 3.6.0 which is incompatible.
azure-cli-core 2.68.0 requires knack~=0.11.0, but you have knack 0.12.0 which is incompatible.
azure-cli-core 2.68.0 requires msal[broker]==1.31.1, but you have msal 1.32.0 which is incompatible.
azure-cli-core 2.68.0 requires msal-extensions==1.2.0, but you have msal-extensions 1.3.1 which is incompatible.
awscli 2.24.27 requires cryptography<43.0.2,>=40.0.0, but you have cryptography 44.0.2 which is incompatible.


In [17]:
from odp.client import OdpClient # The SDK
from shapely.geometry import box, MultiPolygon, Polygon
import pandas as pd
import geopandas as gpd
import json
from shapely import wkt

In [6]:
client = OdpClient()

In [8]:
## Request the dataset from the catalog using the UUID:
proSeas_dataset = client.catalog.get(("3e32fd06-4eb7-4da2-9acb-dd0ecb58aa88"))
proSeas_dataset.metadata.display_name

'ProtectedSeas Navigator - Comprehensive Database of Marine Life Protections'

In [9]:
proSeas_data = client.table_v2(proSeas_dataset)

## Create a geographic query and search for specific MPAs

In [10]:
# Bounding box for Southern Norway
# Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 56
lat_max = 62
lon_min = 8
lon_max = 12
query_geometry = box(lon_min, lat_min, lon_max, lat_max).wkt
query_geometry

'POLYGON ((12 56, 12 62, 8 62, 8 56, 12 56))'

In [61]:
# Find Natura2000 MPAs within the search area
df_geo = pd.concat(proSeas_data.select(f"geometry within '{query_geometry}' AND boundary_source == 'Natura2000'").dataframes(), ignore_index=True)
df_geo.head()

,other_nets,nets,wdpa_id,designation,effective_from,category_name,site_minor_version,construction_prohibited,cast_nets,misc_gear,...,url,gillnetting,regulation_url,purse_seine_roundhaul_surrounding_nets,dredges,dredging_dumping,year_est,purpose,version_start_date,restrictions
0,NaN,3.0,555579659; 555541364,Site of Community Importance (Habitats Directi...,,Other,0,3.0,NaN,3.0,...,Natura2000|https://natura2000.eea.europa.eu/Na...,3.0,Council Directive 92/43/EEC|https://eur-lex.eu...,NaN,NaN,NaN,2000.0,The site is one of the most species-rich and d...,2021-08-03,"According to our research, this site either do..."
1,NaN,3.0,555533409,Site of Community Importance (Habitats Directive),,Other,0,3.0,NaN,3.0,...,Natura2000|https://natura2000.eea.europa.eu/Na...,3.0,Council Directive 92/43/EEC|https://eur-lex.eu...,NaN,NaN,NaN,2011.0,The main purpose of Sundsby is to preserve a v...,2021-08-03,"According to our research, this site either do..."
2,NaN,3.0,555533395,Site of Community Importance (Habitats Directive),None,Other,0,3.0,NaN,3.0,...,Natura2000|https://natura2000.eea.europa.eu/Na...,3.0,Council Directive 92/43/EEC|https://eur-lex.eu...,NaN,NaN,NaN,2011.0,The purpose is to protect two islands with spe...,2021-08-03,"According to our research, this site either do..."
3,NaN,3.0,555541358,Special Protection Area (Birds Directive),,Other,0,3.0,NaN,3.0,...,Natura2000|https://natura2000.eea.europa.eu/Na...,3.0,Directive 2009/147/EC|https://eur-lex.europa.e...,NaN,NaN,NaN,2000.0,The Natura 2000 area Torsviken is the priority...,2021-08-03,"According to our research, this site either do..."
4,NaN,3.0,555557012; 555533442,OSPAR Marine Protected Area; Site of Community...,None,IUCN MPA,0,3.0,NaN,3.0,...,http://mpa.ospar.org/home_ospar/mpa_datasheets...,3.0,https://skyddadnatur.naturvardsverket.se/handl...,NaN,NaN,NaN,2006.0,Gullmarsfjorden is one of the most species-ric...,2021-08-03,1. Professional bivalve dredging is prohibited...


## Get all the data from a country

In [11]:
country = "Norway" # select a country

In [12]:
df_country = pd.concat(proSeas_data.select(f"country == '{country}'").dataframes(), ignore_index=True)
df_country.head()

,geometry_reduced,dredging_prohibited,gillnets_entangling_nets,boundary_source,last_update,dip_scoop_nets,landing_prohibited,season,mooring,spear_fishing,...,url,site_major_version,misc_gear,longlining,traps_n_pots,removal_of_historic_artifacts_prohibited,construction_prohibited,diving_prohibited,recreational_restrictions,gillnetting
0,"MULTIPOLYGON Z (((14.550964 68.413521 0, 14.54...",3.0,NaN,European Environment Agency - CDDA,2021-08-03,NaN,3.0,Year-round,NaN,3.0,...,MPA Website|https://faktaark.naturbase.no/?id=...,1,3.0,3.0,3.0,3.0,2.0,3.0,0.0,3.0
1,"MULTIPOLYGON Z (((5.51676 61.372289 0, 5.51807...",1.0,NaN,European Environment Agency - CDDA,2019-08-14,NaN,2.0,Year-round,NaN,3.0,...,https://faktaark.naturbase.no/?id=VV00001317,1,3.0,3.0,3.0,3.0,1.0,2.0,0.0,3.0
2,"MULTIPOLYGON Z (((11.293291 59.105496 0, 11.29...",3.0,NaN,European Environment Agency - CDDA,2019-04-11,NaN,3.0,Year-round,NaN,3.0,...,https://faktaark.naturbase.no/?id=VV00002784,1,3.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0
3,"MULTIPOLYGON Z (((5.01464 61.313663 0, 5.01561...",1.0,NaN,European Environment Agency - CDDA,2019-08-01,NaN,2.0,Year-round,NaN,3.0,...,https://faktaark.naturbase.no/?id=VV00001316,1,3.0,3.0,3.0,3.0,1.0,3.0,2.0,3.0
4,"MULTIPOLYGON Z (((6.601554 58.086171 0, 6.6021...",1.0,NaN,European Environment Agency - CDDA,2019-04-10,NaN,3.0,Year-round,NaN,3.0,...,https://faktaark.naturbase.no/?id=VV00000474,1,3.0,3.0,3.0,3.0,1.0,3.0,0.0,3.0


## Save it in your prefered file format

### Pandas

In [18]:
# Uncomment the line below to select the output format

# Export to CSV
df_country.to_csv('df_country.csv', index=False)

# Export to JSON
# df_country.to_json('df_country.json', orient='records', lines=True)

# Export to Parquet
# df_country.to_parquet('df_country.parquet', index=False)

### GeoPandas

In [ ]:
# Function to convert WKT string to 2D Shapely geometry
def convert_wkt_to_2d(geometry_wkt):
    try:
        geom = wkt.loads(geometry_wkt)  # Convert text to Shapely geometry
        if geom.has_z:
            # Remove Z by keeping only X, Y coordinates
            return MultiPolygon([
                Polygon([(x, y) for x, y, _ in polygon.exterior.coords])
                for polygon in geom.geoms
            ])
        return geom  # Already 2D
    except Exception as e:
        print(f"Error converting geometry: {e}")
        return None  # Handle invalid geometries gracefully

In [ ]:
# Convert DataFrame to GeoDataFrame
df_country['geometry'] = df_country['geometry'].apply(convert_wkt_to_2d)
gdf = gpd.GeoDataFrame(df_country, geometry='geometry', crs="EPSG:4326")

In [ ]:
gdf.to_file("df_country.geojson", driver="GeoJSON")
print("GeoJSON saved successfully!")

In [15]:
# Save as Shapefile
gdf.to_file("df_country_shapefile.shp", driver='ESRI Shapefile')

print("Shapefile saved successfully!")

/tmp/ipykernel_433/3222584011.py:25: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("df_country_shapefile.shp", driver='ESRI Shapefile')
/srv/conda/envs/notebook/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'geometry_reduced' to 'geometry_r'
  ogr_write(
/srv/conda/envs/notebook/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dredging_prohibited' to 'dredging_p'
  ogr_write(
/srv/conda/envs/notebook/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'gillnets_entangling_nets' to 'gillnets_e'
  ogr_write(
/srv/conda/envs/notebook/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'boundary_source' to 'boundary_s'
  ogr_write(
/srv/conda/envs/notebook/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/l

Shapefile saved successfully!
